In [2]:
import pandas as pd

df = pd.read_csv("data/Seasons_Stats.csv")

df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [3]:
dropped_columns = ["Pos", "Age", "Tm"]

df.drop(dropped_columns,  axis=1, inplace=True)
df = df.fillna(0)
df = df.iloc[: , 1:]

df.head()

,Year,Player,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,63.0,0.0,0.0,0.0,0.368,0.0,0.467,0.0,...,0.705,0.0,0.0,0.0,176.0,0.0,0.0,0.0,217.0,458.0
1,1950.0,Cliff Barker,49.0,0.0,0.0,0.0,0.435,0.0,0.387,0.0,...,0.708,0.0,0.0,0.0,109.0,0.0,0.0,0.0,99.0,279.0
2,1950.0,Leo Barnhorst,67.0,0.0,0.0,0.0,0.394,0.0,0.259,0.0,...,0.698,0.0,0.0,0.0,140.0,0.0,0.0,0.0,192.0,438.0
3,1950.0,Ed Bartels,15.0,0.0,0.0,0.0,0.312,0.0,0.395,0.0,...,0.559,0.0,0.0,0.0,20.0,0.0,0.0,0.0,29.0,63.0
4,1950.0,Ed Bartels,13.0,0.0,0.0,0.0,0.308,0.0,0.378,0.0,...,0.548,0.0,0.0,0.0,20.0,0.0,0.0,0.0,27.0,59.0


In [4]:
import numpy as np
def padding_(seasons, seq_len):
    new_seasons = []
    for season in seasons:
        season = season[:22]
        new_seasons.append(np.pad(season, [(0,seq_len - season.shape[0]), (0,0)], mode='constant'))
    new_seasons = np.array(new_seasons)
    return new_seasons

In [5]:
import torch
import torch.nn as nn
device = "cuda:0"
class GOATnn(nn.Module):

    def __init__(self, batch_size, seq_len = 22, feature_size = 48, drop_prob = 0.35):
        super(GOATnn, self).__init__()
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.feature_size = feature_size
        
        self.linear1 = nn.Linear( feature_size * seq_len, 128)
        self.bnorm1 =  nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(128, 32)
        self.dropout = nn.Dropout(0.2)
        self.linear3 = nn.Linear(32,4)
        self.linear4 = nn.Linear(4, 2)

        self.sig = nn.Softmax()
    def forward(self, input):
        input = input.reshape(self.batch_size, self.feature_size * self.seq_len)
        
        input = self.linear1(input)
        input = self.bnorm1(input)
        input = self.linear2(input)
        input = self.dropout(input)
        input = self.linear3(input)
        input = self.dropout(input)
        input = self.linear4(input)
        return input
   

In [198]:
import torch.optim.lr_scheduler as lr_scheduler
def train_model_era(year, batch_size, goat_tier, df):
    import numpy as np
    grouped = df.groupby(['Player'])

    grouped = list(grouped)
    type(grouped[0])
    X = []
    y = []
    seq_len = 22
    grouped = grouped[1:]

    goat_x = []
    goat_y = []
    ngoat_x = []
    ngoat_y = []
    for i in range(len(grouped)):
        player, player_df = grouped[i]
        player_df.drop(['Player'], axis = 1, inplace=True)
        if player_df['Year'].to_list()[0] < year or player_df['Year'].to_list()[0] >= year + 10:
            continue
        df_np = player_df.to_numpy()
        if player in goat_tier:
            goat_y.append(1)
            goat_x.append(df_np)
        else:
            ngoat_y.append(0)
            ngoat_x.append(df_np)

    goat_x = random_sample(goat_x, len(ngoat_x))
    goat_y = [1 for i in range(len(goat_x))]
    
    X = goat_x + ngoat_x
    y = goat_y + ngoat_y
    y = np.array([y])
    X = padding_(X, seq_len)
    print(X.shape)
    model = GOATnn(batch_size)
    model.to(device)
    lr=1e-5
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = lr_scheduler.LinearLR(optimizer, total_iters=10000)
    from torch.utils.data import TensorDataset, DataLoader
    batch_inputs = X.astype(np.float32)
    batch_outputs = y.T
    train_data = TensorDataset(torch.from_numpy(batch_inputs), torch.from_numpy(batch_outputs))

    # dataloaders

    # make sure to SHUFFLE your data
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    epochs = 10000
    counter = 0
    print_every = 500
    clip = 5
    valid_loss_min = np.Inf

    model.train()
    for i in range(epochs):
        
        for inputs, labels in train_loader:
            counter += 1
            inputs, labels = inputs.to(device), labels.to(device)
            output = model(inputs)
            loss = criterion(output.squeeze(), labels.squeeze())
            optimizer.zero_grad()

            loss.backward()
            
            optimizer.step()
            
            if counter%print_every == 0:
                    
                model.train()
                print("Epoch: {}/{}...".format(i+1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.6f}...".format(loss.item()))
        scheduler.step()
    torch.save(model.state_dict(), str(year))
                


In [208]:
train_model_era(2000, 680, ["LeBron James"], df.copy())

(1360, 22, 48)
Epoch: 250/10000... Step: 500... Loss: 0.566044...
Epoch: 500/10000... Step: 1000... Loss: 0.500547...
Epoch: 750/10000... Step: 1500... Loss: 0.483023...
Epoch: 1000/10000... Step: 2000... Loss: 0.439688...
Epoch: 1250/10000... Step: 2500... Loss: 0.402899...
Epoch: 1500/10000... Step: 3000... Loss: 0.384181...
Epoch: 1750/10000... Step: 3500... Loss: 0.363140...
Epoch: 2000/10000... Step: 4000... Loss: 0.357604...
Epoch: 2250/10000... Step: 4500... Loss: 0.332962...
Epoch: 2500/10000... Step: 5000... Loss: 0.335271...
Epoch: 2750/10000... Step: 5500... Loss: 0.311439...
Epoch: 3000/10000... Step: 6000... Loss: 0.286490...
Epoch: 3250/10000... Step: 6500... Loss: 0.301296...
Epoch: 3500/10000... Step: 7000... Loss: 0.298919...
Epoch: 3750/10000... Step: 7500... Loss: 0.295674...
Epoch: 4000/10000... Step: 8000... Loss: 0.273387...
Epoch: 4250/10000... Step: 8500... Loss: 0.269961...
Epoch: 4500/10000... Step: 9000... Loss: 0.269238...
Epoch: 4750/10000... Step: 9500... 

In [11]:

model = GOATnn(1)

model.cuda()


GOATnn(
  (linear1): Linear(in_features=1056, out_features=128, bias=True)
  (bnorm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=32, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear3): Linear(in_features=32, out_features=4, bias=True)
  (linear4): Linear(in_features=4, out_features=2, bias=True)
  (sig): Softmax(dim=None)
)

In [12]:
def get_players():
    grouped = df.groupby(['Player'])
    players = []

    
    grouped = list(grouped)
    X = []
    seq_len = 22
    grouped = grouped[1:]

    
    for i in range(len(grouped)):
        player, player_df = grouped[i]
        player_df.drop(['Player'], axis = 1, inplace=True)
        
        df_np = player_df.to_numpy()
        X.append(df_np)
        players.append(player)

  
    X = padding_(X, seq_len)
    print(X.shape)
   
    batch_inputs = X.astype(np.float32)
    return batch_inputs, players

In [40]:
def get_players_era(start, end):
    grouped = df.groupby(['Player'])
    players = []

    
    grouped = list(grouped)
    X = []
    seq_len = 22
    grouped = grouped[1:]

    
    for i in range(len(grouped)):
        player, player_df = grouped[i]
        player_df.drop(['Player'], axis = 1, inplace=True)
        if player_df['Year'].to_list()[0] < start or player_df['Year'].to_list()[0] >=end :
            continue
        df_np = player_df.to_numpy()
        X.append(df_np)
        players.append(player)

  
    X = padding_(X, seq_len)
    print(X.shape)
   
    batch_inputs = X.astype(np.float32)
    return batch_inputs, players

In [46]:
models = ["2000"]
batch_size = 1
model.batch_size = 1
player_score = []
batch_inputs, players = get_players_era(2000, 2020)
player_score = [0 for i in range(len(players))]
model.eval()
for model_name in models:
    model.load_state_dict(torch.load(model_name))
    cplayer_score = []
    index = 0
    for batch_input in batch_inputs:
        probs = model(torch.from_numpy(batch_input).cuda())
        player_score[index] += torch.nn.functional.softmax(probs)[0][1].item()
        index += 1


(1274, 22, 48)


<ipython-input-46-d53594d6744e>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  player_score[index] += torch.nn.functional.softmax(probs)[0][1].item()


In [47]:
players

['A.J. Bramlett',
 'A.J. Guyton',
 'A.J. Hammons',
 'A.J. Price',
 'Aaron Brooks',
 'Aaron Gordon',
 'Aaron Gray',
 'Aaron Harrison',
 'Aaron Miles',
 'Acie Law',
 'Adam Harrington',
 'Adam Morrison',
 'Adonis Thomas',
 'Adreian Payne',
 'Adrian Griffin',
 'Al Horford',
 'Al Jefferson',
 'Al Thornton',
 'Al-Farouq Aminu',
 'Alan Anderson',
 'Alan Williams',
 'Alando Tucker',
 'Alec Burks',
 'Aleksandar Radojevic',
 'Alex Abrines',
 'Alex Acker',
 'Alex Garcia',
 'Alex Kirk',
 'Alex Len',
 'Alex Poythress',
 'Alex Scales',
 'Alex Stepheson',
 'Alexander Johnson',
 'Alexey Shved',
 'Alexis Ajinca',
 'Allan Ray',
 'Allen Crabbe',
 'Alonzo Gee',
 'Alton Ford',
 'Alvin Jones',
 "Amar'e Stoudemire",
 'Amir Johnson',
 'Anderson Varejao',
 'Andray Blatche',
 'Andre Barrett',
 'Andre Brown',
 'Andre Dawkins',
 'Andre Drummond',
 'Andre Emmett',
 'Andre Iguodala',
 'Andre Miller',
 'Andre Owens',
 'Andre Roberson',
 'Andrea Bargnani',
 'Andreas Glyniadakis',
 'Andrei Kirilenko',
 'Andres Nocioni

In [48]:
list1, list2 = (list(x) for x in zip(*sorted(zip(player_score, players), reverse=True,key=lambda pair: pair[0])))

In [49]:
for i in range(len(list1)):
    print(list1[i], list2[i])

0.9346023797988892 LeBron James
0.8121815919876099 Dwyane Wade
0.7987475991249084 Carmelo Anthony
0.7032292485237122 Lamar Odom
0.7030085325241089 Monta Ellis
0.6958621144294739 Elton Brand
0.6754509806632996 Chris Bosh
0.6638768911361694 Dwight Howard
0.6626642346382141 Tony Parker
0.6508166790008545 Andre Miller
0.619691789150238 Carlos Boozer
0.6189677715301514 Richard Hamilton
0.6165530681610107 Kevin Durant
0.6092897653579712 Pau Gasol
0.606971800327301 Rudy Gay
0.589297890663147 Amar'e Stoudemire
0.5867475867271423 Chris Paul
0.5842716693878174 Jason Terry
0.5770631432533264 Joe Johnson
0.569970428943634 Jason Richardson
0.5617181658744812 LaMarcus Aldridge
0.5590239763259888 Deron Williams
0.5536375641822815 Luol Deng
0.5481337308883667 Russell Westbrook
0.53657466173172 Josh Smith
0.5301807522773743 David West
0.5154595375061035 Andre Iguodala
0.49548575282096863 Thaddeus Young
0.49511823058128357 Stephen Curry
0.47941723465919495 Baron Davis
0.46958011388778687 Al Jefferson
0.

In [50]:
list2.index("Stephen Curry")

28